In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_excel(r'/kaggle/input/website-urls-for-web-scraping-and-data-analysis/blackcoffer websites link dataset.xlsx')
df

# Extract Title and Text from articles by using Requests and BeautifulSoup-

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
title_list=[]
main_text=[]
for i in df['URL']:
    try:
        response=requests.get(i)
        soup=BeautifulSoup(response.text, 'html.parser')
        title_list.append(soup.find('h1').text)
        inner_text=[]
        for j in soup.find_all('p')[16:-3]:
            inner_text.append(j.text)
        main_text.append(' '.join(inner_text))
    except:
        title_list.append(np.nan)
        main_text.append(np.nan)
df['Title']=title_list
df['Text']=main_text

In [ ]:
df_copy=df.copy()

# **covert all characters in lowercase-**

In [ ]:

df_copy['Text']=df_copy['Text'].astype('str').apply(lambda x:' '.join(x.lower() for x in x.split()))

In [ ]:
df_copy

# **removing punctuation**

In [ ]:
import string
string.punctuation

In [ ]:
# df_copy['Text']=df_copy['Text'].apply(lambda x: ''.join(re.findall(r'[a-z+" "]', x)))

In [ ]:
df_copy

# **removing stopwords-**

In [ ]:
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# allstopwords=stopwords.words('english')
# df['Text'].apply(lambda x: " ".join(i for i in x.split() if i not in allstopwords))
file_list=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if os.path.join(dirname, filename).endswith('.txt'):
            file_list.append(os.path.join(dirname, filename))
print(file_list)

In [ ]:
all_stopwords=[]
for i in file_list: 
    try:
        f=open(i, encoding='latin8')
        all_stopwords.append(f.read())
        f.close()
    except:
        print('ERROR')

In [ ]:
df_copy['Text'] = df_copy['Text'].apply(lambda x: " ".join(i for i in x.split() if i not in all_stopwords))   # remove all stopwords

In [ ]:
df_copy

# **Lemmatization-**

In [ ]:
import textblob
from textblob import Word

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
df_copy['Text']=df_copy['Text'].apply(lambda text: ' '.join([Word(word).lemmatize() for word in text.split()]))

In [ ]:
df_copy

# **Tokenization-**

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

**Avg_sent_len_column-**

In [ ]:
average_sent_len=[]

for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)

    sent_tok=nltk.sent_tokenize(x)
    
    ave_sent_len=len(word_tok)/len(sent_tok)
    
    if ave_sent_len==1.0:
        average_sent_len.append(np.nan)
    else:
        average_sent_len.append(ave_sent_len)
        
df_copy['AVG_SENTENCE_LENGTH']=average_sent_len
df_copy

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=df_copy['Text']
countvectorizer= CountVectorizer()
X=countvectorizer.fit_transform(corpus)
result=X.toarray()
print(result)

In [ ]:
df1=pd.DataFrame(data=result, columns=countvectorizer.get_feature_names_out()) 
df1

In [ ]:
df_copy

**complex_word_percentage-**

In [ ]:
per_of_word=0
per_of_words=[]
for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)
    no_of_word=len(word_tok)
    vow='aeiou'
    no_of_com_word=0
    for w in word_tok:
        vowel=0
        for v in vow:
            if v in w:
                vowel+=1
        if vowel>1:
            no_of_com_word+=1
    if no_of_com_word==0:
        per_of_words.append(np.nan)
    else:
        per_of_word=(no_of_com_word/no_of_word)*100
        per_of_words.append(per_of_word)
#     print(per_of_word)
df_copy['complex_word_percentage']=per_of_words

In [ ]:
df_copy

**Fog_index-**
o.4(avg_sent_len+10*(per_complex_count))

In [ ]:
fog_ind=0
fog_indx=[]
for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)
    no_of_word=len(word_tok)
    sent_tok=nltk.sent_tokenize(x)
    ave_sent_len=len(word_tok)/len(sent_tok)
    vow='aeiou'
    no_of_com_word=0
    for w in word_tok:
        vowel=0
        for v in vow:
            if v in w:
                vowel+=1
        if vowel>1:
            no_of_com_word+=1
    per_of_word=no_of_com_word/no_of_word
    fog_ind=0.4*(ave_sent_len+per_of_word)
    if ave_sent_len==1.0 and per_of_word==0:
        fog_indx.append(np.nan)
    else:
        fog_indx.append(fog_ind)

In [ ]:
df_copy['Fog_Index']=fog_indx

**AVG NUMBER OF WORDS PER SENTENCE-**

In [ ]:
average_sent_len=[]

for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)

    sent_tok=nltk.sent_tokenize(x)
    
    ave_sent_len=len(word_tok)/len(sent_tok)
    
    if ave_sent_len==1.0:
        average_sent_len.append(np.nan)
    else:
        average_sent_len.append(ave_sent_len)
        
df_copy['AVG_WORD_PER_SENT']=average_sent_len

**COMPLEX WORD COUNT**

In [ ]:
per_of_word=0
com_words=[]
for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)
    no_of_word=len(word_tok)
    vow='aeiou'
    no_of_com_word=0
    for w in word_tok:
        vowel=0
        for v in vow:
            if v in w:
                vowel+=1
        if vowel>1:
            no_of_com_word+=1
    if no_of_com_word==0:
        com_words.append(np.nan)
    else:
        
        com_words.append(no_of_com_word)
#     print(per_of_word)
df_copy['COMPLEX_WORD_COUNT']=com_words

**WORD COUNT-**

In [ ]:
word_counts=[]
for i in range(len(df_copy)):
    x=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', x))
    word_tok=nltk.word_tokenize(punc_word)
    word_count=len(word_tok)
    word_counts.append(word_count)
df_copy['WORD_COUNT']=word_counts

**SYLLABLE PER WORD-**

In [ ]:
##from scratch
# def syllable_count(word):
#     word = word.lower()
#     count = 0
#     vowels = "aeiouy"
#     if word[0] in vowels:
#         count += 1
#     for index in range(1, len(word)):
#         if word[index] in vowels and word[index - 1] not in vowels:
#             count += 1
#     if word.endswith("e"):
#         count -= 1
#     if count == 0:
#         count += 1
#     return count

# print(syllable_count('HAIRY'))

In [ ]:
pip install syllables

In [ ]:
syllable_list=[]
import syllables
for i in range(len(df_copy)):
    text=df_copy['Text'][i]
    vowels='aeiouy'
    no_of_syllables=0
    punc_word=''.join(re.findall(r'[a-z+" "]', text))
    word_tok=nltk.word_tokenize(punc_word)
    for i in word_tok:
        syllable_count=syllables.estimate(i)
        no_of_syllables+=syllable_count
    syllable_list.append(no_of_syllables)
df_copy['SYLLABLE_COUNT']=syllable_list

**PERSONAL PRONOUNS-**

In [ ]:
pronoun_list=[]
pronoun=["i", "you", "he", "she", "it", "we", "they", "them", "us", "him", "her", "his", "hers", "its", "theirs", "our", "your"]
for i in range(len(df_copy)):
    text=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', text))
    word_tok=nltk.word_tokenize(punc_word)
    pronoun_count=0
    for i in word_tok:
        if i in pronoun:
            pronoun_count+=1
    pronoun_list.append(pronoun_count)
df_copy['PERSONAL_PRONOUN']=pronoun_list

**AVG WORD LENGTH-**

In [ ]:
avg_words_len=[]
for i in range(len(df_copy)):
    text=df_copy['Text'][i]
    punc_word=''.join(re.findall(r'[a-z+" "]', text))
    word_tok=nltk.word_tokenize(punc_word)
    no_of_word=len(word_tok)
    no_of_char=0
    for i in word_tok:
        count_per_word=len(i)
        no_of_char+=count_per_word
    avg_word_len=no_of_char/no_of_word
    avg_words_len.append(avg_word_len)
df_copy['AVG_WORD_LEN']=avg_words_len  

In [ ]:
pip install vaderSentiment


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# neg_score=[]
# pos_score=[]
# pol_score=[]
# for i in range(len(df_copy)):
#     text=df_copy['Text'][i]
#     punc_word=''.join(re.findall(r'[a-z+" "]', text))
#     word_tok=nltk.word_tokenize(punc_word)
#     neg,pos,pol=0,0,0
#     for i in word_tok:
#         sid_obj = SentimentIntensityAnalyzer()

#         neg += sid_obj.polarity_scores(i)['neg']
#         pos += sid_obj.polarity_scores(i)['pos']
#         pol += sid_obj.polarity_scores(i)['compound']

#     neg_score.append(neg)
#     pos_score.append(pos)
#     pol_score.append(pol)

In [ ]:
df_copy